In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

# List of first names and last names
first_names = ["Ali", "Zahra", "Reza", "Sara", "Mohammad", "Fatemeh", "Hossein", "Maryam", "Mehdi", "Narges", "Hamed", "Roya"]
last_names = ["Ahmadi", "Hosseini", "Karimi", "Rahimi", "Hashemi", "Ebrahimi", "Moradi", "Mohammadi", "Rostami", "Fazeli", "Hosseinzadeh", "Niknam"]

# Set a seed for reproducibility
random.seed(42)
np.random.seed(42)

# Generate 24 unique random names
random_names = set()
while len(random_names) < 24:
    first_name = random.choice(first_names)
    last_name = random.choice(last_names)
    random_names.add(f"{first_name} {last_name}")

random_names = list(random_names)  # Convert set to list

# Function to generate a relationship matrix
def generate_relationship_matrix():
    relationship_matrix = np.zeros((24, 24), dtype=int)
    conflict_pairs = set()
    while len(conflict_pairs) < 40:
        i = random.randint(0, 23)
        j = random.randint(0, 23)
        if i != j and (i, j) not in conflict_pairs and (j, i) not in conflict_pairs:
            conflict_pairs.add((i, j))
    for i, j in conflict_pairs:
        relationship_matrix[i, j] = 1
    conflict_matrix = relationship_matrix + relationship_matrix.T
    np.fill_diagonal(conflict_matrix, 0)
    return conflict_matrix

# Generate 1000 relationship matrices
relationship_matrices = [generate_relationship_matrix() for _ in range(1000)]
relationship_matrices_tensor = torch.tensor(relationship_matrices, dtype=torch.float)

# Define the seating grid dimensions
ROWS, COLS = 4, 6

# Define the loss function
def loss_function(arrangement, relationship_matrix):
    arrangement = arrangement.view(ROWS, COLS)
    loss = torch.tensor(0.0, dtype=torch.float, requires_grad=True)
    for r in range(ROWS):
        for c in range(COLS):
            person = arrangement[r, c].item()
            if r > 0:  # Check the person above
                if relationship_matrix[person, arrangement[r - 1, c].item()] == 1:
                    loss = loss + 1
            if r < ROWS - 1:  # Check the person below
                if relationship_matrix[person, arrangement[r + 1, c].item()] == 1:
                    loss = loss + 1
            if c > 0:  # Check the person to the left
                if relationship_matrix[person, arrangement[r, c - 1].item()] == 1:
                    loss = loss + 1
            if c < COLS - 1:  # Check the person to the right
                if relationship_matrix[person, arrangement[r, c + 1].item()] == 1:
                    loss = loss + 1
    return loss

# Define the neural network with three hidden layers and dropout for regularization
class SeatingNN(nn.Module):
    def __init__(self):
        super(SeatingNN, self).__init__()
        self.fc1 = nn.Linear(24, 512)  # First hidden layer with 128 units
        self.fc2 = nn.Linear(512, 256)  # Second hidden layer with 64 units
        self.fc3 = nn.Linear(256, 64)   # Third hidden layer with 32 units
        self.fc4 = nn.Linear(64, 24)   # Output layer
        self.dropout = nn.Dropout(0.5) # Dropout layer with 50% dropout rate

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU activation after first hidden layer
        x = self.dropout(x)          # Dropout after first hidden layer
        x = torch.relu(self.fc2(x))  # ReLU activation after second hidden layer
        x = self.dropout(x)          # Dropout after second hidden layer
        x = torch.relu(self.fc3(x))  # ReLU activation after third hidden layer
        x = self.fc4(x)              # Output layer
        return x

# Training the model with learning rate scheduler
def train_model(relationship_matrices, num_epochs=2000, batch_size=20, learning_rate=0.01):
    model = SeatingNN()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.5)
    criterion = loss_function

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        
        # Sample a batch of relationship matrices
        batch_indices = np.random.choice(len(relationship_matrices), batch_size, replace=False)
        batch_matrices = relationship_matrices[batch_indices]
        
        # Create an initial random arrangement
        arrangement = torch.randperm(24, dtype=torch.float).unsqueeze(0).requires_grad_(True)
        
        # Forward pass
        output = model(arrangement)
        
        # Ensure output is within valid index range by applying argsort
        sorted_indices = output.squeeze(0).argsort(dim=0)
        
        # Calculate loss
        loss = torch.tensor(0.0, dtype=torch.float)
        for matrix in batch_matrices:
            loss += criterion(sorted_indices, matrix)
        
        loss /= batch_size  # Average loss over the batch
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, LR: {scheduler.get_last_lr()[0]:.6f}')

    # Get the final arrangement
    with torch.no_grad():
        final_output = model(torch.randperm(24, dtype=torch.float).unsqueeze(0))
        final_arrangement = final_output.squeeze(0).argsort(dim=0).detach().numpy()
    return final_arrangement

# Function to map numbers to names and reshape
def map_numbers_to_names(arrangement, names_list, rows, cols):
    mapped_arrangement = np.array([names_list[i] for i in arrangement])
    return mapped_arrangement.reshape(rows, cols)

# Train the model and get the optimal arrangement
optimal_arrangement = train_model(relationship_matrices_tensor)

# Map the numbers to names and reshape
mapped_arrangement = map_numbers_to_names(optimal_arrangement, random_names, ROWS, COLS)

print("Optimal arrangement with names:")
print(mapped_arrangement)


Epoch [100/2000], Loss: 10.5000, LR: 0.010000
Epoch [200/2000], Loss: 11.5000, LR: 0.010000
Epoch [300/2000], Loss: 9.1000, LR: 0.010000
Epoch [400/2000], Loss: 11.1000, LR: 0.010000
Epoch [500/2000], Loss: 10.7000, LR: 0.005000
Epoch [600/2000], Loss: 11.1000, LR: 0.005000
Epoch [700/2000], Loss: 10.3000, LR: 0.005000
Epoch [800/2000], Loss: 10.4000, LR: 0.005000
Epoch [900/2000], Loss: 10.9000, LR: 0.005000
Epoch [1000/2000], Loss: 12.8000, LR: 0.002500
Epoch [1100/2000], Loss: 12.5000, LR: 0.002500
Epoch [1200/2000], Loss: 11.1000, LR: 0.002500
Epoch [1300/2000], Loss: 10.8000, LR: 0.002500
Epoch [1400/2000], Loss: 11.7000, LR: 0.002500
Epoch [1500/2000], Loss: 9.4000, LR: 0.001250
Epoch [1600/2000], Loss: 11.2000, LR: 0.001250
Epoch [1700/2000], Loss: 10.0000, LR: 0.001250
Epoch [1800/2000], Loss: 10.6000, LR: 0.001250
Epoch [1900/2000], Loss: 12.3000, LR: 0.001250
Epoch [2000/2000], Loss: 11.3000, LR: 0.000625
Optimal arrangement with names:
[['Roya Hosseinzadeh' 'Sara Ebrahimi' '